In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.integrate as integrate
import scipy.special as special
import math
from matplotlib.colors import ListedColormap

In [2]:
def T_phi_iso(r, rb = 250, **kwargs):
    return r/r 

In [3]:
# Tphi for nfw potential
def T_phi_nfw_pot(r,rb = 1, c = 10, **kwargs):
    # defining variables
    rs = rb/c
    M_nfw_b = math.log(1+c) - c/(1+c)
    # create x array of shape and proportional to r
    x = c*r
    # create array with with shape of x
    M_nfw = np.array(x)
    # assign value to each element in M_nfw
    for i in range(0,len(x)):
        M_nfw[i] = math.log(1+x[i])-x[i]/(1+x[i])
    # assigning variable name
    new = (M_nfw/r)/(M_nfw_b/rb)
    # find maximum value
    maxM = np.max((M_nfw/r)/(M_nfw_b/rb))
    # if the value reaches the max, the function is kept at max
    for i in reversed(range(len(new))):
        if new[i-1] < new[i] :
            # reassign element values
            new[i-1] = new[i]
    # return adjusted array
    return new

In [32]:
# Tphi for nfw profile
def T_phi_nfw(r,rb = 1, c = 10, **kwargs):
    rs = rb/c
    x = c*r
    M_nfw_b = math.log(1+c) - c/(1+c)
    M_nfw = np.array(x)
    for i in range(0,len(x)):
        M_nfw[i] = math.log(1+x[i])-x[i]/(1+x[i])
    # f_Tx = math.log(1+x)/x - 1/(1+x) which is basically M_nfw/x
    # f_Tc = math.log(1+c)/c - 1/(1+c)
    # f_T is prop to M_nfw/r is prop to Tphi
    # T_phi = T_phi(rb)*f_Tx/f_Tc
    return (M_nfw/r)/(M_nfw_b/rb)

In [33]:
def Tb_mass(Mb = 1e12*1.989e30, **kwargs): # unit management
    G = 6.67e-11*(1/3.086e+19)**3 #kpc^3/kg/s^2
    k = 1.38e-23*(1/3.086e+19)**2 # boltzmann constant in W/(kpc^2*K)
    H0 = 70 * 3.24078e-20 #km/s/Mpc * Mpc/km
    Hz = H0 # could be a parameter
    mmp = 1.0e-27 # mean mass per particle
    #Mb = 4*np.pi*rb**3/3*200*3*Hz**2/(8*np.pi*G) # rearange for rb
    p = 3*Hz**2/(8*np.pi*G)
    rb = (3*Mb/(4*np.pi*200*p))**(1/3) #kpc
    vc = (G*Mb/rb)**(1/2)#*1/3.24078e-17 # km/s
    #Mb = (4*np.pi*rb**3/3*200*p)
    #Tphi = G*Mb*mmp/(2*k*rb) # k is boltzmann's constant, rb?
    Tb = vc**2 * mmp/(2*k)
    # for milky way solar masses, temp should be 10^6K
    return Tb

In [34]:
Tb_mass(1e12*1.989e30)

755259.834888052

In [37]:
# imports the necessary library/modules for the graph with sliders
from ipywidgets import interact,interact_manual

# integration
def integral_function(K0 = 0.0, pwr = 4/3, Mb = 1e12, T_phi = T_phi_nfw, Tb = Tb_mass, **kwargs):
    # constants
    n = 1000 # number of intervals
    rmin = 0.01
    rb = 1
    rmax = rb
    Krb = 1 #K0 + K1
    K1 = Krb - K0
    

    ln_rmin = math.log(rmin)
    ln_r = np.zeros((n))

    for i in range(0, n):
        ln_r[i] = ln_rmin+i/(n-1)*math.log(rmax/rmin)
    ln_r
    r = np.exp(ln_r)

    dln_r = math.log(rmax/rmin)/(n-1)
    K = K1*(r/rb)**pwr + K0 # K(r)
    
    # defines Tphi based on default/selection
    Tphi = T_phi(r) #*Tb_mass()
    #Tb = np.max(T_phi(r))
        
    d_dlnr = -4/5*Tphi/(K**(3/5)) 

    # initialize arrays
    Ti_Ki = np.zeros((n))
    
    # This is where the issue was
    Ti_Ki[-1] = 4/(3*pwr)*Tphi[-1]   #Tb/Krb**(3/5)
    print(Ti_Ki[-1])
    for j in range(1,n):
        i = n-j


        Ti_Ki[i-1] = Ti_Ki[i] - d_dlnr[i]*dln_r

    Tr = Ti_Ki*(K**(3/5))

    sns.set()

    cmap1 = 'PiYG' #ListedColormap(sns.color_palette("ch:s=.25,rot=-.25"))
    cmap2 = 'magma' #ListedColormap(sns.color_palette("ch:s=-.2,r=.6"))
    
    colors = Tr
    colors2 = Tphi
    # change
    plt.scatter(r,Tr, c = colors, cmap=cmap1, label = 'Tr')
    plt.colorbar()
    plt.scatter(r,Tphi, c = colors2, cmap=cmap2, label = 'Tphi')
    plt.colorbar()
    plt.semilogx()
    plt.legend()
    
    
    #plt.show()
    plt.xlabel('ln_r')

    plt.ylabel('Tr')
    
    # I DID IT IM PROUD
    
    #if np.any(Tr) < 2 :
    #    plt.ylim((0,2)) 
        

    sns.set_context('notebook')

In [38]:
interact(integral_function, K0=(0.0,1, .01), pwr=(0.01,2,0.01), Mb=(1e10,1e14,1e10), T_phi = [("NFW", T_phi_nfw), ("ISO", T_phi_iso), ("NFW Potential", T_phi_nfw_pot)], Tb = [("MASS", Tb_mass)]);


interactive(children=(FloatSlider(value=0.0, description='K0', max=1.0, step=0.01), FloatSlider(value=1.333333…

**Geneva Archer**